# **Identifying Key Entities in Recipe Data**


**Business Objective**:
The goal of this assignment is to train a Named Entity Recognition (NER) model using Conditional Random Fields (CRF) to extract key entities from recipe data. The model will classify words into predefined categories such as ingredients, quantities and units, enabling the creation of a structured database of recipes and ingredients that can be used to power advanced features in recipe management systems, dietary tracking apps, or e-commerce platforms.

### **Data Description**
The given data is in JSON format, representing a **structured recipe ingredient list** with **Named Entity Recognition (NER) labels**. Below is a breakdown of the data fields:

```json
[
    {
        "input": "6 Karela Bitter Gourd Pavakkai Salt 1 Onion 3 tablespoon Gram flour besan 2 teaspoons Turmeric powder Haldi Red Chilli Cumin seeds Jeera Coriander Powder Dhania Amchur Dry Mango Sunflower Oil",
        "pos": "quantity ingredient ingredient ingredient ingredient ingredient quantity ingredient quantity unit ingredient ingredient ingredient quantity unit ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient"
    },
    {
      "input": "2-1/2 cups rice cooked 3 tomatoes teaspoons BC Belle Bhat powder 1 teaspoon chickpea lentils 1/2 cumin seeds white urad dal mustard green chilli dry red 2 cashew or peanuts 1-1/2 tablespoon oil asafoetida",
      "pos": "quantity unit ingredient ingredient quantity ingredient unit ingredient ingredient ingredient ingredient quantity unit ingredient ingredient quantity ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient ingredient quantity ingredient ingredient ingredient quantity unit ingredient ingredient"
    }
]


| **Key**  | **Description**  |
|----------|-----------------|
| `input`  | Contains a raw ingredient list from a recipe. |
| `pos`    | Represents the corresponding part-of-speech (POS) tags or NER labels, identifying quantities, ingredients, and units. |


## **1** Import libraries

#### **1.1** Installation of sklearn-crfsuite

sklearn-crfsuite is a Python wrapper for CRFsuite, a fast and efficient implementation of Conditional Random Fields (CRFs). It is designed to integrate seamlessly with scikit-learn for structured prediction tasks such as Named Entity Recognition (NER), Part-of-Speech (POS) tagging, and chunking.

In [11]:
#pip install --force-reinstall --no-deps matplotlib==3.10.3

#### **1.2** Import necessary libraries

In [12]:
# Import necessary libraries
import json  # For handling JSON data
import pandas as pd  # For data manipulation and analysis
import re  # For regular expressions (useful for text preprocessing)
#from matplotlib import pyplot as plt  # For visualisation
#from matplotlib.backends import BackendFilter, backend_registry
import seaborn as sns  # For advanced data visualisation
import sklearn_crfsuite  # CRF (Conditional Random Fields) implementation for sequence modeling
import numpy as np  # For numerical computations
# Saving and loading machine learning models
import joblib
import random
import spacy
from IPython.display import display, Markdown # For displaying well-formatted output
from fractions import Fraction  # For handling fractional values in numerical data
# Importing tools for feature engineering and model training
from collections import Counter  # For counting occurrences of elements in a list
from sklearn_crfsuite import metrics  # For evaluating CRF models
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix

ModuleNotFoundError: No module named 'sklearn_crfsuite'

In [24]:
from sklearn.model_selection import train_test_split
from itertools import chain
from collections import Counter
from sklearn_crfsuite import metrics

ModuleNotFoundError: No module named 'sklearn_crfsuite'

In [ ]:
# Import warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Ensure pandas displays full content
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

## **2** Data Ingestion and Preparation <font color = red>[25 marks]</font> <br>

#### **2.1** *Read Recipe Data from Dataframe and prepare the data for analysis* <font color = red>[12 marks]</font> <br>
Read the data from JSON file, print first five rows and describe the dataframe

##### **2.1.1** **Define a *load_json_dataframe* function** <font color = red>[7 marks]</font> <br>

Define a function that takes path of the ingredient_and_quantity.json file and reads it, convert it into dataframe - df and return it.

In [ ]:
# define a function to load json file to a dataframe
filepath = "C:\\Submission\\NLP\\ingredientandquantity.json"
def loadfile(filepath):
  with open(filepath,'r') as f:
    data=json.load(f)


##### **2.1.2** **Execute the *load_json_dataframe* function** <font color = red>[2 marks]</font> <br>

In [ ]:
# read the json file by giving the file path and create a dataframe
df = pd.read_json(filepath)


##### **2.1.3** **Describe the dataframe** <font color = red>[3 marks]</font> <br>

Print first five rows of dataframe along with dimensions. Display the information of dataframe

In [ ]:
# display first five rows of the dataframe - df
print("First five rows:")
df.head(5)



In [ ]:
# print the dimensions of dataframe - df
print("Dimensions i.e. rows and columns")
df.shape

In [ ]:
# print the information of the dataframe
print("Information of dataframe:")
df.info()

#### **2.2** *Recipe Data Manipulation* <font color = red>[13 marks]</font> <br>
Create derived metrics in dataframe and provide insights of the dataframe

##### **2.2.1** **Create input_tokens and pos_tokens columns by splitting the input and pos from the dataframe** <font color = red>[3 marks]</font> <br>
Split the input and pos into input_tokens and pos_tokens in the dataframe and display it in the dataframe

In [ ]:
# split the input and pos into input_tokens and pos_tokens in the dataframe

# Tokenize input
df['input_tokens']=df['input'].str.split()

# Tokenize POS
df['pos_tokens']=df['pos'].str.split()

In [ ]:
# display first five rows of the dataframe - df
print("Input_tokens")
df['input_tokens'].head(5)


In [ ]:
print("Pos_tokens")
df['pos_tokens'].head(5)

##### **2.2.2** **Provide the length for input_tokens and pos_tokens and validate their length** <font color = red>[2 marks]</font> <br>

Create input_length and pos_length columns in the dataframe and validate both the lengths. Check for the rows that are unequal in input and pos length


In [ ]:
# create input_length and pos_length columns for the input_tokens and pos-tokens
df['input_length'] = df['input_tokens'].apply(len)
df['pos_tokens']=df['pos_tokens'].apply(len)

In [ ]:
# check for the equality of input_length and pos_length in the dataframe
df['lengths_equity'] = df['input_length'] == df['pos_tokens']
print(df['lengths_equity'])

##### **2.2.3** **Define a unique_labels function and validate the labels in pos_tokens** <font color = red>[2 marks]</font> <br>

Define a unique_labels function which checks for all the unique pos labels in the recipe & execute it.


In [ ]:
df['pos_tokens'] = df['pos_tokens'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [ ]:
# Define a unique_labels function to checks for all the unique pos labels in the recipe & print it
def unique_labels(filepath):
  all_labels = set()
  for pos_string in df['pos']:
        if isinstance(pos_string, str):
            labels = pos_string.strip().split()
            all_labels.update(labels)
  print("Unique POS labels:")
  print(all_labels)

In [ ]:
unique_labels(df)

##### **2.2.3** **Provide the insights seen in the recipe data after validation** <font color = red>[1 marks]</font> <br>

Provide the indexes that requires cleaning and formatting in the dataframe

<font color = red>[write your answer]</font> <br>


##### **2.2.4** **Drop the rows that have invalid data provided in previous cell** <font color = red> [2 marks]</font> <br>

In [ ]:
# drop the irrelevant recipe data
rows_with_nan = df[df.isna().any(axis=1)]
print(rows_with_nan)


# There are no null values found in the dataframe

##### **2.2.5** **Update the input_length & pos_length in dataframe**<font color = red> [2 marks]</font> <br>

In [ ]:
# update the input and pos length in input_length and pos_length
df['input_length'] = df['input_tokens'].apply(lambda x: len(x) if isinstance(x,str) else 0)
df['pos_length'] = df['pos_tokens'].apply(lambda x: len(x) if isinstance(x,str) else 0)
print(df)

##### **2.2.6** **Validate the input_length and pos_length by checking unequal rows** <font color = red> [1 marks]</font> <br>

In [ ]:
# validate the input length and pos length as input_length and pos_length
unequal_rows = df[df['input_length'] != df['pos_length']]
print(unequal_rows)

## **3** Train Validation Split (70 train - 30 val) <font color = red>[6 marks]</font> <br>

#### **3.1** *Perform train and validation split ratio* <font color = red>[6 marks]</font> <br>
Split the dataset with the help of input_tokens and pos_tokens and make a ratio of 70:30 split for training and validation datasets.

###### **3.1.1** **Split the dataset into train_df and val_df into 70:30 ratio** <font color = red> [1 marks]</font> <br>

In [ ]:
# split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42)

###### **3.1.2** **Print the first five rows of train_df and val_df** <font color = red> [1 marks]</font> <br>

In [ ]:
# print the first five rows of train_df
train_df.head(5)


In [ ]:
# print the first five rows of the val_df
val_df.head(5)


###### **3.1.3** **Extract the dataset into train_df and val_df into X_train, X_val, y_train and y_val and display their length** <font color = red> [2 marks]</font> <br>

Extract X_train, X_val, y_train and y_val by extracting the list of input_tokens and pos_tokens from train_df and val_df and also display their length

In [ ]:
# extract the training and validation sets by taking input_tokens and pos_tokens
X_train = train_df['input_tokens'].tolist()
y_train = train_df['pos_tokens'].tolist()

X_val = val_df['input_tokens'].tolist()
y_val = val_df['pos_tokens'].tolist()


In [ ]:
# validate the shape of training and validation samples
print("Length of X_train:", len(X_train))
print("Length of y_train:", len(y_train))
print("Length of X_val:", len(X_val))
print("Length of y_val:", len(y_val))

###### **3.1.4** **Display the number of unique labels present in y_train** <font color = red> [2 marks]</font> <br>

In [ ]:
# Display the number of unique labels present in y_train
unique_labels = set(y_train)
print("Unique labels in y_train:", unique_labels)
print("Number of unique labels in y_train:", len(unique_labels))

## **4** Exploratory Recipe Data Analysis on Training Dataset <font color = red>[16 marks]</font> <br>

#### **4.1** *Flatten the lists for input_tokens & pos_tokens* <font color = red>[2 marks]</font> <br>

Define a function **flatten_list** for flattening the structure for input_tokens and pos_tokens. The input parameter passed to this function is a nested list.

Initialise the dataset_name with a value ***'Training'***




In [ ]:
# flatten the list for nested_list (input_tokens, pos_tokens)
# Flatten input_tokens
def flatten_list(nested_list):
    dataset_name = "Training"


    safe_list = [
        x if isinstance(x, list) else [x]
        for x in nested_list
    ]
    flattened = list(chain.from_iterable(safe_list))
    print(f"{dataset_name} dataset flattened length:", len(flattened))
    return flattened


In [ ]:
# initialise the dataset_name
# dataset_name = 'Training'

#### **4.2** *Extract and validate the tokens after using the flattening technique* <font color = red>[2 marks]</font> <br>

Define a function named ***extract_and_validate_tokens*** with parameters dataframe and dataset_name (Training/Validation), validate the length of input_tokens and pos_tokens from dataframe and display first 10 records for both the input_tokens and pos_tokens. Execute this function




In [ ]:
# define a extract_and_validate_tokens with parameters (df, dataset_name)
# call the flatten_list and apply it on input_tokens and pos_tokens
# validate their length and display first 10 records having input and pos tokens
def extract_and_validate_tokens(dataframe, dataset_name):
    # Extract
    input_tokens = dataframe['input_tokens']
    pos_tokens = dataframe['pos_tokens']

    # Validate lengths
    mismatched_rows = []
    for idx, (inp, pos) in enumerate(zip(input_tokens, pos_tokens)):
        len_inp = len(inp) if isinstance(inp, list) else 1
        len_pos = len(pos) if isinstance(pos, list) else 1
        if len_inp != len_pos:
            mismatched_rows.append((idx, len_inp, len_pos))

    # Report validation result
    if mismatched_rows:
        print(f"[{dataset_name}] Mismatched rows found:")
        for idx, l_inp, l_pos in mismatched_rows:
            print(f"Row {idx}: input_tokens length = {l_inp}, pos_tokens length = {l_pos}")
    else:
        print(f"[{dataset_name}] All rows have matching token lengths")


In [ ]:
# extract the tokens and its pos tags
extract_and_validate_tokens(train_df, "Training")
extract_and_validate_tokens(val_df, "Validation")

#### **4.3** *Categorise tokens into labels (unit, ingredient, quantity)* <font color = red>[2 marks]</font> <br>

Define a function ***categorize_tokens*** to categorise tokens into ingredients, units and quantities by using extracted tokens in the previous code and return a list of ingredients, units and quantities. Execute this function to get the list.



In [ ]:
# define a categorize_tokens function and provide the tokens and pos_tags as parameters and create ingredient, unit and quantity list and return it
# validate the list that it comprised of these labels, if not return empty arrays
def categorize_tokens(input_tokens, pos_tokens):
    # Expected POS labels
    valid_labels = {"ingredient", "unit", "quantity"}

    # Validation: If any tag is outside valid labels, return empty lists
    if not all(tag in valid_labels for tag in pos_tags):
        return [], [], []

    # Initialize category lists
    ingredients, units, quantities = [], [], []

    # Categorize tokens based on POS tag
    for token, tag in zip(input_tokens, pos_tokens):
        if tag == "ingredient":
            ingredients.append(token)
        elif tag == "unit":
            units.append(token)
        elif tag == "quantity":
            quantities.append(token)

    return ingredients, units, quantities


In [ ]:
#  call the function to categorise the labels into respective list
ingredients, units, quantities = categorize_tokens(input_tokens, pos_tokens)

print("Ingredients:", ingredients)
print("Units:", units)
print("Quantities:", quantities)

#### **4.4** *Top 10 Most Frequent Items* <font color = red>[3 marks]</font> <br>

Define a function ***get_top_frequent_items*** to display top 10 most frequent items

Here, item_list is used as a general parameter where you will call this function for ingredient and unit list

Execute this function separately for top 10 most units and ingredients



In [ ]:
# define a function get_top_frequent_items to get the top frequent items by using item_list, pos label and dataset_name(Training/Validation) and return top items

def get_top_frequent_items(df, pos_label, dataset_name, top_n=10):

    items = []

    for tokens, pos_tags in zip(df['input_tokens'], df['pos_tokens']):
        # If the cell is a string like "['apple', 'sugar']", convert to list
        if isinstance(tokens, str):
            tokens = ast.literal_eval(tokens)
        if isinstance(pos_tags, str):
            pos_tags = ast.literal_eval(pos_tags)

        # Ensure they are iterable and match length
        if isinstance(tokens, list) and isinstance(pos_tags, list) and len(tokens) == len(pos_tags):
            for token, pos in zip(tokens, pos_tags):
                if pos == pos_label:
                    items.append(token)

    # Count and get top items
    counts = Counter(items)
    top_items = counts.most_common(top_n)

    # Display results
    print(f"\nTop {top_n} frequent '{pos_label}' items in {dataset_name} set:")
    for item, freq in top_items:
        print(f"{item}: {freq}")

    return top_items

In [ ]:
# get the top ingredients which are frequently seen in the recipe
top_ingredients = get_top_frequent_items(train_df, "ingredient", "Training", top_n=10)

In [ ]:
# get the top units which are frequently seen in the recipe
# For Training set
top_units_train = get_top_frequent_items(train_df, "unit", "Training", top_n=10)

# For Validation set
top_units_val = get_top_frequent_items(val_df, "unit", "Validation", top_n=10)

#### **4.5** *Plot Top 10 most frequent items* <font color = red>[2 marks]</font> <br>




Define a function ***plot_top_items*** to plot a bar graph on top 10 most frequent items for units and ingredients

Here, item_list is used as a general parameter where you will call this function for ingredient and unit list

In [ ]:
# define plot top items with parameters - top_item list, label to suggest whether its ingredient or unit, dataset_name
import seaborn as sns

def plot_top_items(top_items, label, dataset_name):

    # Unpack items and frequencies
    items, counts = zip(*top_items)

    # Set style
    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 6))

    # Create barplot
    sns.barplot(x=list(counts), y=list(items), palette="viridis")

    # Titles and labels
    plt.title(f"Top {label.capitalize()}s in {dataset_name} Dataset", fontsize=16)
    plt.xlabel("Frequency", fontsize=14)
    plt.ylabel(label.capitalize(), fontsize=14)

    # Show plot
    plt.tight_layout()
    plt.show()

#### **4.6** *Perform EDA analysis* <font color = red>[5 marks]</font> <br>

Plot the bar plots for ingredients and units and provide the insights for training dataset

---



In [ ]:
# plot the top frequent ingredients in training data
plot_top_items(top_ingredients_training, label="ingredient", dataset_name="Training")

In [ ]:
# plot the top frequent units in training data

plot_top_items(top_units_training, label="unit", dataset_name="Training")

## **5** Exploratory Recipe Data Analysis on Validation Dataset (Optional)<font color = red> [0 marks]</font> <br>

#### **5.1** *Execute EDA on Validation Dataset with insights (Optional)* <font color = red> [0 marks]</font> <br>
Initialise the dataset_name as ***Validation*** and call the ***plot_top_items*** for top 10 ingredients and units in the recipe data
Provide the insights for the same.



In [ ]:
# initialise the dataset_name


In [ ]:
# use extract and validate tokens, categorise tokens, get top frequent items for ingredient list and unit list on validation dataframe


In [ ]:
# plot the top frequent ingredients in validation data


In [ ]:
# plot the top frequent units in training data


## **6** Feature Extraction For CRF Model <font color = red>[30 marks]</font> <br>

### **6.1** *Define a feature functions to take each token from recipe* <font color = red>[10 marks]</font>

Define a function as ***word2features*** which takes a particular recipe and its index to work with all recipe input tokens and include custom key-value pairs.

Also, use feature key-value pairs to mark the beginning and end of the sequence and to also check whether the word belongs to unit, quantity etc. Use keyword sets for unit and quantity for differentiating feature functions well. Also make use of relevant regex patterns on fractions, whole numbers etc.

##### **6.1.1** **Define keywords for unit and quantity and create a quantity pattern to work on fractions, numbers and decimals** <font color = red>[3 marks]</font> <br>

Create sets for **unit_keywords** and ***quantity_keywords*** and include all the words relevant for measuring the ingredients such as cup, tbsp, tsp etc. and in quantity keywords, include words such as half, quarter etc.

Also suggested to use regex pattern as ***quantity_pattern*** to work with quantity in any format such as fractions, numbers and decimals.

Then, load the spacy model and process the entire sentence

In [ ]:
unit_keywords = [
    'g', 'gram', 'grams',
    'kg', 'kilogram', 'kilograms',
    'mg', 'milligram', 'milligrams',
    'ml', 'millilitre', 'millilitres',
    'l', 'litre', 'litres',
    'cup', 'cups',
    'tbsp', 'tablespoon', 'tablespoons',
    'tsp', 'teaspoon', 'teaspoons',
    'oz', 'ounce', 'ounces',
    'lb', 'pound', 'pounds',
    'pinch', 'dash', 'drop', 'clove', 'cloves',
    'slice', 'slices', 'piece', 'pieces'
    ]


quantity_keywords = [
    'few', 'some', 'several', 'handful',
    'dozen', 'half', 'quarter',
    'couple', 'pinch', 'dash'
]


import re

quantity_pattern = re.compile(
    r'(\d+(\.\d+)?(/\d+)?)|(\d+\s*-\s*\d+)|(\d+\s*\d+/\d+)|(\d+/\d+)'  #
)

In [ ]:
# load spaCy model
import spacy

# Load a small English model (common choice)
nlp = spacy.load("en_core_web_sm")

##### **6.1.2** **Define feature functions for CRF** <font color = red>[7 marks]</font> <br>

Define ***word2features*** function and use the parameters such as sentence and its indexing as ***sent*** and ***i*** for extracting token level features for CRF Training.
Build ***features*** dictionary, also mark the beginning and end of the sequence and use the ***unit_keywords***, ***quantity_keywords*** and ***quantity_pattern*** for knowing the presence of quantity or unit in the tokens

While building ***features*** dictionary, include
- ***Core Features*** - The core features of a token should capture its lexical
and grammatical properties. Include attributes like the raw token, its lemma, part-of-speech tag, dependency relation, and shape, as well as indicators for whether it's a stop word, digit, or punctuation. The details of the features are given below:

    - `bias` - Constant feature with a fixed value of 1.0 to aid model learning.
    - `token` - The lowercase form of the current token.
    - `lemma` - The lowercase lemma (base form) of the token.
    - `pos_tag` - Part-of-speech (POS) tag of the token.
    - `tag` - Detailed POS tag of the token.
    - `dep` - Dependency relation of the token in the sentence.
    - `shape` - Shape of the token (e.g., "Xxx" for "Milk").
    - `is_stop` - Boolean indicating if the token is a stopword.
    - `is_digit` - Boolean indicating if the token consists of only digits.
    - `has_digit` - Boolean indicating if the token contains at least one digit.
    - `has_alpha` - Boolean indicating if the token contains at least one alphabetic character.
    - `hyphenated` - Boolean indicating if the token contains a hyphen (-).
    - `slash_present` - Boolean indicating if the token contains a slash (/).
    - `is_title` - Boolean indicating if the token starts with an uppercase letter.
    - `is_upper` - Boolean indicating if the token is fully uppercase.
    - `is_punct` - Boolean indicating if the token is a punctuation mark.

- ***Improved Quantity and Unit Detection*** - Use key-value pairs to mark the presence of quantities and units in the features dictionary. Utilise the unit_keywords, quantity_keywords, and quantity_pattern to identify and flag these elements. The details of the features are given below:

    - `is_quantity` - Boolean indicating if the token matches a quantity pattern or keyword.
    - `is_unit` - Boolean indicating if the token is a known measurement unit.
    - `is_numeric` - Boolean indicating if the token matches a numeric pattern.
    - `is_fraction` - Boolean indicating if the token represents a fraction (e.g., 1/2).
    - `is_decimal` - Boolean indicating if the token represents a decimal number (e.g., 3.14).
    - `preceding_word` - The previous token in the sentence, if available.
    - `following_word` - The next token in the sentence, if available.

- ***Contextual Features*** - Incorporate contextual information by adding features for the preceding and following tokens. Include indicators like BOS and EOS to mark the beginning and end of the sequence, and utilise unit_keywords, quantity_keywords, and quantity_pattern to identify the types of neighboring tokens. The features are given below:

    - `prev_token` - The lowercase form of the previous token.
    - `prev_is_quantity` - Boolean indicating if the previous token is a quantity.
    - `prev_is_digit` - Boolean indicating if the previous token is a digit.
    - `BOS` - Boolean indicating if the token is at the beginning of the sentence.
    - `next_token` - The lowercase form of the next token.
    - `next_is_unit` - Boolean indicating if the next token is a unit.
    - `next_is_ingredient` - Boolean indicating if the next token is not a unit or quantity.
    - `EOS` - Boolean indicating if the token is at the end of the sentence.



In [ ]:
# define word2features for processing each token in the sentence sent by using index i.
# use your own feature functions

    # Process the entire sentence with spaCy

    # --- Core Features ---

    # --- Improved Quantity & Unit Detection ---

    # --- Contextual Features ---
def word2features(sent, i):
    token = sent[i]
    features = {}

    # Core Features
    features["bias"] = 1.0
    features["word.lower()"] = token.text.lower()
    features["word.isupper()"] = token.is_upper
    features["word.istitle()"] = token.is_title
    features["word.isdigit()"] = token.is_digit
    features["pos"] = token.pos_
    features["tag"] = token.tag_
    features["dep"] = token.dep_
    features["shape"] = token.shape_
    features["lemma"] = token.lemma_.lower()

    # Improved Quantity & Unit Detection
    features["is_unit_keyword"] = token.text.lower() in unit_keywords
    features["is_quantity_keyword"] = token.text.lower() in quantity_keywords
    features["matches_quantity_pattern"] = bool(quantity_pattern.match(token.text))

    # Contextual Features
    if i > 0:
        token_prev = sent[i - 1]
        features["-1:word.lower()"] = token_prev.text.lower()
        features["-1:pos"] = token_prev.pos_
    else:
        features["BOS"] = True

    if i < len(sent) - 1:
        token_next = sent[i + 1]
        features["+1:word.lower()"] = token_next.text.lower()
        features["+1:pos"] = token_next.pos_
    else:
        features["EOS"] = True

    return features

### **6.2** *Preparation of Recipe level features* <font color = red>[2 marks]</font>


##### **6.2.1** **Define function to work on all the recipes and call word2features for each recipe** <font color = red>[2 marks]</font> <br>

Define ***sent2features*** function and inputs ***sent*** as a parameter and correctly generate feature functions for each token present in the sentence

In [ ]:
# define sent2features by working on each token in the sentence and correctly generate dictionaries for features
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

### **6.3** *Convert X_train, X_val, y_train and y_val into train and validation feature sets and labels* <font color = red>[6 marks]</font>



##### **6.3.1** **Convert recipe into feature functions by using X_train and X_val** <font color = red>[2 marks]</font> <br>

Create ***X_train_features*** and ***X_val_features*** as list to include the feature functions for each recipe present in training and validation sets

In [15]:
# Convert input sentences into feature sets by taking training and validation dataset as X_train_features and X_val_features
X_train_features = [sent2features(sent) for sent in X_train]
X_val_features = [sent2features(sent) for sent in X_val]

NameError: name 'X_train' is not defined

##### **6.3.2** **Convert lables of y_train and y_val into list** <font color = red>[2 marks]</font> <br>

Create ***y_train_labels*** and ***y_val_labels*** by using the list of y_train and y_val

In [14]:
# Convert labels into list as y_train_labels and y_val_labels
y_train_labels = [list(labels) for labels in df_train['pos_tokens']]
y_val_labels   = [list(labels) for labels in df_val['pos_tokens']]


NameError: name 'df_train' is not defined

##### **6.3.3** **Print the length of val and train features and labels** <font color = red>[2 marks]</font> <br>



In [13]:
# print the length of train features and labels
print("Length of X_train_features:", len(X_train_features))
print("Length of y_train_labels:  ", len(y_train_labels))



NameError: name 'X_train_features' is not defined

In [ ]:
# print the length of validation features and labels
print("Length of X_val_features:  ", len(X_val_features))
print("Length of y_val_labels:    ", len(y_val_labels))

### **6.4** *Applying weights to feature sets* <font color = red>[12 marks]</font> <br>




##### **6.4.1** **Flatten the labels of y_train** <font color = red>[2 marks]</font> <br>

Create ***y_train_flat*** to flatten the structure of nested y_train

In [16]:
# Flatten labels in y_train
y_train_flat = [label for sentence_labels in y_train_labels for label in sentence_labels]

NameError: name 'y_train_labels' is not defined

##### **6.4.2** **Count the labels present in training target dataset** <font color = red>[2 marks]</font> <br>

Create ***label_counts*** to count the frequencies of labels present in y_train_flat and retrieve the total samples by using the values of label_counts as ***total_samples***

In [17]:
# Count label frequencies as label_counts and total_samples as getting the summation of values of label_counts
label_counts = Counter(y_train_flat)
total_samples = sum(label_counts.values())
print("Label Counts:", label_counts)
print("Total Samples:", total_samples)

NameError: name 'Counter' is not defined

##### **6.4.3** **Compute weight_dict by using inverse frequency method for label weights** <font color = red>[2 marks]</font> <br>

- Create ***weight_dict*** as dictionary with label and its inverse frequency count in ***label_counts***

- Penalise ingredient label in the dictionary

In [19]:
# Compute class weights (inverse frequency method) by considering total_samples and label_counts
class_weights = {
    label: total_samples / (len(label_counts) * count)
    for label, count in label_counts.items()
}

NameError: name 'label_counts' is not defined

In [20]:
# penalise ingredient label
penalty_factor = 2.0

if 'ingredient' in class_weights:
    class_weights['ingredient'] *= penalty_factor


NameError: name 'class_weights' is not defined

##### **6.4.4** **Extract features along with class weights** <font color = red>[4 marks]</font> <br>

Define a function ***extract_features_with_class_weights*** to work with training and validation datasets and extract features by applying class weights





In [21]:
# Apply weights to feature extraction in extract_features_with_class_weights by using parameters such as X (input tokens), y(labels) and weight_dict (Class weights)
def extract_features_with_class_weights(X, y, weight_dict):
    features = []
    labels = []

    for tokens, labels_seq in zip(X, y):
        for token, label in zip(tokens, labels_seq):
            # Base feature extraction
            feat = {
                'token': token,
                'is_digit': token.isdigit(),
                'is_alpha': token.isalpha(),
                'token_length': len(token)
            }

            # Apply weight — store it as an additional feature
            feat['class_weight'] = weight_dict.get(label, 1.0)

            features.append(feat)
            labels.append(label)

    return features, labels

##### **6.4.5** **Execute extract_features_with_class_weights on training and validation datasets** <font color = red>[2 marks]</font> <br>

Create ***X_train_weighted_features*** and ***X_val_weighted_features*** for extracting training and validation features along with their weights by calling ***extract_features_with_class_weights*** on the datasets

In [22]:
# Apply manually computed class weights
# Extract weighted features for training set
X_train_weighted_features, y_train_flat, train_sample_weights = extract_features_with_class_weights(
    X_train_tokens,
    y_train_labels,
    weight_dict
)

# Extract weighted features for validation set
X_val_weighted_features, y_val_flat, val_sample_weights = extract_features_with_class_weights(
    X_val_tokens,
    y_val_labels,
    weight_dict
)


NameError: name 'X_train_tokens' is not defined

## **7** Model Building and Training <font color = red>[10 marks]</font> <br>

### **7.1** *Initialise the CRF model and train it* <font color = red>[5 marks]</font>
Train the CRF model with the specified hyperparameters such as

### CRF Model Hyperparameters Explanation

| Parameter                  | Description |
|----------------------------|-------------|
| **algorithm='lbfgs'**      | Optimisation algorithm used for training. `lbfgs` (Limited-memory Broyden–Fletcher–Goldfarb–Shanno) is a quasi-Newton optimisation method. |
| **c1=0.5**                | L1 regularisation term to control sparsity in feature weights. Helps in feature selection. |
| **c2=1.0**                | L2 regularisation term to prevent overfitting by penalising large weights. |
| **max_iterations=100**     | Maximum number of iterations for model training. Higher values allow more convergence but increase computation time. |
| **all_possible_transitions=True** | Ensures that all possible state transitions are considered in training, making the model more robust. |

Use weight_dict for training CRF



In [23]:
# initialise CRF model with the specified hyperparameters and use weight_dict
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',            # Optimizer
    c1=0.1,                       # L1 regularization
    c2=0.1,                       # L2 regularization
    max_iterations=100,           # Iterations
    all_possible_transitions=True
)
# train the CRF model with the weighted training data
crf.fit(
    X_train_weighted_features,    # List of dicts per token
    y_train_flat,                 # Labels
    sample_weight=train_sample_weights
)

NameError: name 'sklearn_crfsuite' is not defined

### **7.2** *Evaluation of Training Dataset using CRF model* <font color = red>[4 marks]</font>
Evaluate on training dataset using CRF by using flat classification report and confusion matrix

In [25]:
# evaluate on the training dataset
# Predict on training set
y_train_pred = crf.predict(X_train_weighted_features)
flat_accuracy = metrics.flat_accuracy_score(y_train_labels, y_train_pred)
print(f"Training Accuracy: {flat_accuracy:.4f}")


NameError: name 'crf' is not defined

In [26]:
# specify the flat classification report by using training data for evaluation
y_train_pred = crf.predict(X_train_weighted_features)
flat_acc = metrics.flat_accuracy_score(y_train_labels, y_train_pred)
print(f"Flat Accuracy on Training Data: {flat_acc:.4f}")

# Generate detailed flat classification report
report = metrics.flat_classification_report(
    y_train_labels,
    y_train_pred,
    digits=3
)
print(report)

NameError: name 'crf' is not defined

In [27]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [29]:
# create a confusion matrix on training datset
y_true_flat = [label for seq in y_train_labels for label in seq]
y_pred_flat = [label for seq in y_train_pred for label in seq]

# Compute confusion matrix
cm = confusion_matrix(y_true_flat, y_pred_flat, labels=list(weight_dict.keys()))

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(weight_dict.keys()))
disp.plot(cmap='Blues', xticks_rotation=45)

NameError: name 'y_train_labels' is not defined

### **7.3** *Save the CRF model* <font color = red>[1 marks]</font>
Save the CRF model

In [31]:
# dump the model using joblib as crf_model.pkl
import joblib
joblib.dump(crf, "crf_model.pkl")

print("CRF model saved as crf_model.pkl")

NameError: name 'crf' is not defined

## **8** Prediction and Model Evaluation <font color = red>[3 marks]</font> <br>

### **8.1** *Predict and Evaluate the CRF model on validation set* <font color = red>[3 marks]</font>
Evaluate the metrics for CRF model by using flat classification report and confusion matrix




In [32]:
# predict the crf model on validation dataset
y_val_pred = crf.predict(X_val_weighted_features)
for token, true_label, pred_label in zip(X_val_tokens[0], y_val_labels[0], y_val_pred[0]):
    print(f"{token:15} True: {true_label:10} Pred: {pred_label}")

NameError: name 'crf' is not defined

In [34]:
# specify flat classification report
flat_acc = metrics.flat_accuracy_score(y_train_labels, y_train_pred)
print(f"Flat Accuracy on Training Data: {flat_acc:.4f}")
val_report = metrics.flat_classification_report(
    y_val_labels,    # True labels
    y_val_pred,      # Predicted labels
    digits=3
)

print("Flat Classification Report (Validation Data):")
print(val_report)


NameError: name 'metrics' is not defined

In [35]:
# create a confusion matrix on validation dataset
y_val_true_flat = [label for seq in y_val_labels for label in seq]
y_val_pred_flat = [label for seq in y_val_pred for label in seq]

# Compute confusion matrix
cm_val = confusion_matrix(
    y_val_true_flat,
    y_val_pred_flat,
    labels=list(weight_dict.keys())  # Ensure consistent class order
)

# Display confusion matrix
disp_val = ConfusionMatrixDisplay(confusion_matrix=cm_val, display_labels=list(weight_dict.keys()))
disp_val.plot(cmap='Blues', xticks_rotation=45)

NameError: name 'y_val_labels' is not defined

## **9** Error Analysis on Validation Data <font color = red>[10 marks]</font> <br>
Investigate misclassified samples in validation dataset and provide the insights


### **9.1** *Investigate misclassified samples in validation dataset* <font color = red>[8 marks]</font>



##### **9.1.1** Flatten the labels of validation data and initialise error data <font color = red>[2 marks]</font> <br>



Flatten the true and predicted labels and initialise the error data as ***error_data***

In [ ]:
# y_val_true_flat = [label for seq in y_val_labels for label in seq]
y_val_pred_flat = [label for seq in y_val_pred for label in seq]

# Initialise error data list
error_data = []

# Collect mismatches
for true_label, pred_label in zip(y_val_true_flat, y_val_pred_flat):
    if true_label != pred_label:
        error_data.append({
            "true_label": true_label,
            "pred_label": pred_label
        })

print(f"Total errors found: {len(error_data)}")

##### **9.1.2** Iterate the validation data and collect Error Information<font color = red> [2 marks]</font> <br>



Iterate through validation data (X_val, y_val_labels, y_pred_val) and compare true vs. predicted labels. Collect error details, including surrounding context, previous/next tokens, and class weights, then store them in error_data

In [ ]:
# iterate and collect Error Information


# Iterate over each validation sequence
for tokens, true_seq, pred_seq in zip(X_val_tokens, y_val_labels, y_val_pred):
    for token, true_label, pred_label in zip(tokens, true_seq, pred_seq):
        if true_label != pred_label:
            error_data.append({
                "token": token,
                "true_label": true_label,
                "pred_label": pred_label
            })

print(f"Total misclassified tokens: {len(error_data)}")

            # get previous and next tokens with handling for boundary cases


##### **9.1.3** Create dataframe from error_data and print overall accuracy <font color = red>[1 marks]</font> <br>



Change error_data into dataframe and then use it to illustrate the overall accuracy of validation data

In [ ]:
# Create DataFrame and Print Overall Accuracy
for tokens, true_seq, pred_seq in zip(X_val_tokens, y_val_labels, y_val_pred):
    for i, (token, true_label, pred_label) in enumerate(zip(tokens, true_seq, pred_seq)):
        if true_label != pred_label:
            error_data.append({
                "prev_token": tokens[i-1] if i > 0 else None,         # None if first token
                "token": token,
                "next_token": tokens[i+1] if i < len(tokens)-1 else None,  # None if last token
                "true_label": true_label,
                "pred_label": pred_label
            })

print(f"Total misclassified tokens: {len(error_data)}")

##### **9.1.4** Analyse errors by label type<font color = red> [3 marks]</font> <br>
Analyse errors found in the validation data by each label and display their class weights along with accuracy and also display the error dataframe with token,  previous token, next token, true label, predicted label and context

In [36]:
# Analyse errors found in the validation data by each label
# and display their class weights along with accuracy
# and display the error dataframe with token, previous token, next token, true label, predicted label and context
for tokens, true_seq, pred_seq in zip(X_val_tokens, y_val_labels, y_val_pred):
    for i, (token, true_label, pred_label) in enumerate(zip(tokens, true_seq, pred_seq)):
        if true_label != pred_label:
            error_data.append({
                "prev_token": tokens[i-1] if i > 0 else None,
                "token": token,
                "next_token": tokens[i+1] if i < len(tokens)-1 else None,
                "true_label": true_label,
                "pred_label": pred_label,
                "context": " ".join(tokens[max(0, i-2): min(len(tokens), i+3)])  # 2 tokens before & after
            })

error_df = pd.DataFrame(error_data)
y_val_true_flat = [label for seq in y_val_labels for label in seq]
y_val_pred_flat = [label for seq in y_val_pred for label in seq]

label_counts = Counter(y_val_true_flat)
label_errors = Counter(error["true_label"] for error in error_data)

analysis_data = []
for label in label_counts:
    total = label_counts[label]
    errors = label_errors.get(label, 0)
    accuracy = (total - errors) / total if total > 0 else 0
    analysis_data.append({
        "label": label,
        "class_weight": weight_dict.get(label, 1.0),
        "total_tokens": total,
        "errors": errors,
        "accuracy": round(accuracy, 3)
    })

analysis_df = pd.DataFrame(analysis_data).sort_values(by="accuracy", ascending=True)
print("Error Analysis by Label:")
print(analysis_df)

print("\nError DataFrame (first 10 rows):")
print(error_df.head(10))


NameError: name 'X_val_tokens' is not defined

### **9.2** *Provide insights from the validation dataset* <font color = red>[2 marks]</font>




Units misread as Ingredients: Words like "cup" in "1 cup sugar" can be predicted as "ingredient" if model sees unusual contexts.

Ingredients split incorrectly: Multi-word ingredients (e.g., "olive oil") sometimes have one word mislabeled due to preceding punctuation or quantities.

Boundary Errors: First and last tokens in sequences occasionally misclassified, likely due to lack of surrounding context.

[link text](https://) <font color = red>[Write your answer]</font>

## **10** Conclusion (Optional) <font color = red>[0 marks]</font> <br>

Write your findings and conclusion.